In [1]:
import stats as st
import plotly.graph_objects as go

### Read data

In [2]:
# suite = "gap"
# suite = "lcf"
suite = "cp-spec"

cp_df = st.parse_champsim_output(f"../temp/cp-data")
wp_df = st.parse_champsim_output(f"../temp/wp-data")
wpa_df = st.parse_champsim_output(f"../temp/wpa-data")

### Default runs

In [ ]:
cp_default_df = st.group_by(cp_df, "default")
wp_default_df = st.group_by(wp_df, "default")
wpa_default_df = st.group_by(wpa_df, "default")

# Calculate the means
cp_default_df = st.calculate_means(cp_default_df)
wp_default_df = st.calculate_means(wp_default_df)
wpa_default_df = st.calculate_means(wpa_default_df)

### Instruction Footprint

In [ ]:
# Compute instruction footprint in MB for CP and WP
cp_instr_footprint = cp_default_df["instr_foot_print"] * 64 / 1024**2
wp_instr_footprint = wp_default_df["instr_foot_print"] * 64 / 1024**2
wpa_instr_footprint = wpa_default_df["instr_foot_print"] * 64 / 1024**2

# Compute the percentage difference
percentage_diff_wp_cp = (
    (wp_instr_footprint - cp_instr_footprint) / cp_instr_footprint
) * 100

percentage_diff_wpa_cp = (
    (wpa_instr_footprint - cp_instr_footprint) / cp_instr_footprint
) * 100

# Drop gmean
percentage_diff_wp_cp = percentage_diff_wp_cp.drop("gmean")
percentage_diff_wpa_cp = percentage_diff_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=percentage_diff_wp_cp.index,  # Benchmarks on the x-axis
        y=percentage_diff_wp_cp,  # Percentage difference
        name="WP",
        legendgroup="WP",
    )
)

fig.add_trace(
    go.Bar(
        x=percentage_diff_wpa_cp.index,  # Benchmarks on the x-axis
        y=percentage_diff_wpa_cp,  # Percentage difference
        name="WPA",
        legendgroup="WPA",
    )
)

fig.update_layout(
    title="Instruction Footprint Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Instruction Footprint Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### Instruction Overhead

In [ ]:
# Compute the instruction overhead due to wrong path execution
wp_overhead = (wp_default_df["wrong_path_insts_executed"] / 100_000_000) * 100
wpa_overhead = (wpa_default_df["wrong_path_insts_executed"] / 100_000_000) * 100

# Drop gmean
wp_overhead = wp_overhead.drop("gmean")
wpa_overhead = wpa_overhead.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=wp_overhead.index,  # Benchmarks on the x-axis
        y=wp_overhead,  # Percentage of instructions executed
        name="WP",  # Add legend name
        legendgroup="WP",
    )
)

fig.add_trace(
    go.Bar(
        x=wpa_overhead.index,  # Benchmarks on the x-axis
        y=wpa_overhead,  # Percentage of instructions executed
        name="WPA",  # Add legend name
        legendgroup="WPA",
    )
)

# Update layout for better readability
fig.update_layout(
    title="Instruction Overhead Due to Wrong Path (Aware) Execution",
    yaxis_title="Instruction Overhead (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### IPC

In [ ]:
# Compute the IPC
cp_ipc = cp_default_df["IPC"]
wp_ipc = wp_default_df["IPC"]
wpa_ipc = wpa_default_df["IPC"]

# Drop amean
cp_ipc = cp_ipc.drop("amean")
wp_ipc = wp_ipc.drop("amean")
wpa_ipc = wpa_ipc.drop("amean")

# Calculate speedup
wp_speedup = ((wp_ipc / cp_ipc) - 1) * 100
wpa_speedup = ((wpa_ipc / cp_ipc) - 1) * 100

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=wp_speedup.index,  # Benchmarks on the x-axis
        y=wp_speedup,  # Speedup
        name="WP",  # Add legend name
        legendgroup="WP",
    )
)

fig.add_trace(
    go.Bar(
        x=wpa_speedup.index,  # Benchmarks on the x-axis
        y=wpa_speedup,  # Speedup
        name="WPA",  # Add legend name
        legendgroup="WPA",
    )
)

# Update layout for better readability
fig.update_layout(
    title="Speedup Due to Wrong Path (Aware) Support",
    yaxis_title="Speedup (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### L1I Hits & Misses

In [ ]:
cp_l1i_hits = cp_default_df["L1I_TOTAL_HITS"]
wp_l1i_hits = wp_default_df["L1I_TOTAL_HITS"]
wpa_l1i_hits = wpa_default_df["L1I_TOTAL_HITS"]

cp_l1i_misses = cp_default_df["L1I_TOTAL_MISS"]
wp_l1i_misses = wp_default_df["L1I_TOTAL_MISS"]
wpa_l1i_misses = wpa_default_df["L1I_TOTAL_MISS"]

# Compute the % change in L1I hits and misses
l1i_hits_wp_cp = ((wp_l1i_hits - cp_l1i_hits) / cp_l1i_hits) * 100
l1i_misses_wp_cp = ((wp_l1i_misses - cp_l1i_misses) / cp_l1i_misses) * 100

l1i_hits_wpa_cp = ((wpa_l1i_hits - cp_l1i_hits) / cp_l1i_hits) * 100
l1i_misses_wpa_cp = ((wpa_l1i_misses - cp_l1i_misses) / cp_l1i_misses) * 100

# Drop gmean
l1i_hits_wp_cp = l1i_hits_wp_cp.drop("gmean")
l1i_misses_wp_cp = l1i_misses_wp_cp.drop("gmean")

l1i_hits_wpa_cp = l1i_hits_wpa_cp.drop("gmean")
l1i_misses_wpa_cp = l1i_misses_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=l1i_hits_wp_cp.index,  # Benchmarks on the x-axis
        y=l1i_hits_wp_cp,  # Percentage difference
        name="WP Hits",
        legendgroup="WP Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l1i_hits_wpa_cp.index,  # Benchmarks on the x-axis
        y=l1i_hits_wpa_cp,  # Percentage difference
        name="WPA Hits",
        legendgroup="WPA Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l1i_misses_wp_cp.index,  # Benchmarks on the x-axis
        y=l1i_misses_wp_cp,  # Percentage difference
        name="WP Misses",
        legendgroup="WP Misses",
    )
)

fig.add_trace(
    go.Bar(
        x=l1i_misses_wpa_cp.index,  # Benchmarks on the x-axis
        y=l1i_misses_wpa_cp,  # Percentage difference
        name="WPA Misses",
        legendgroup="WPA Misses",
    )
)

fig.update_layout(
    title="L1I Cache Hits and Misses Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### L1D Hits & Misses

In [ ]:
cp_l1d_hits = cp_default_df["L1D_TOTAL_HITS"]
wp_l1d_hits = wp_default_df["L1D_TOTAL_HITS"]
wpa_l1d_hits = wpa_default_df["L1D_TOTAL_HITS"]

cp_l1d_misses = cp_default_df["L1D_TOTAL_MISS"]
wp_l1d_misses = wp_default_df["L1D_TOTAL_MISS"]
wpa_l1d_misses = wpa_default_df["L1D_TOTAL_MISS"]

# Compute the % change in L1I hits and misses
l1d_hits_wp_cp = ((wp_l1d_hits - cp_l1d_hits) / cp_l1d_hits) * 100
l1d_misses_wp_cp = ((wp_l1d_misses - cp_l1d_misses) / cp_l1d_misses) * 100

l1d_hits_wpa_cp = ((wpa_l1d_hits - cp_l1d_hits) / cp_l1d_hits) * 100
l1d_misses_wpa_cp = ((wpa_l1d_misses - cp_l1d_misses) / cp_l1d_misses) * 100

# Drop gmean
l1d_hits_wp_cp = l1d_hits_wp_cp.drop("gmean")
l1d_misses_wp_cp = l1d_misses_wp_cp.drop("gmean")

l1d_hits_wpa_cp = l1d_hits_wpa_cp.drop("gmean")
l1d_misses_wpa_cp = l1d_misses_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=l1d_hits_wp_cp.index,  # Benchmarks on the x-axis
        y=l1d_hits_wp_cp,  # Percentage difference
        name="WP Hits",
        legendgroup="WP Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l1d_hits_wpa_cp.index,  # Benchmarks on the x-axis
        y=l1d_hits_wpa_cp,  # Percentage difference
        name="WPA Hits",
        legendgroup="WPA Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l1d_misses_wp_cp.index,  # Benchmarks on the x-axis
        y=l1d_misses_wp_cp,  # Percentage difference
        name="WP Misses",
        legendgroup="WP Misses",
    )
)

fig.add_trace(
    go.Bar(
        x=l1d_misses_wpa_cp.index,  # Benchmarks on the x-axis
        y=l1d_misses_wpa_cp,  # Percentage difference
        name="WPA Misses",
        legendgroup="WPA Misses",
    )
)

fig.update_layout(
    title="L1D Cache Hits and Misses Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### L2C Hits & Misses

In [ ]:
cp_l2c_hits = cp_default_df["L2C_TOTAL_HITS"]
wp_l2c_hits = wp_default_df["L2C_TOTAL_HITS"]
wpa_l2c_hits = wpa_default_df["L2C_TOTAL_HITS"]

cp_l2c_misses = cp_default_df["L2C_TOTAL_MISS"]
wp_l2c_misses = wp_default_df["L2C_TOTAL_MISS"]
wpa_l2c_misses = wpa_default_df["L2C_TOTAL_MISS"]

# Compute the % change in L1I hits and misses
l2c_hits_wp_cp = ((wp_l2c_hits - cp_l2c_hits) / cp_l2c_hits) * 100
l2c_misses_wp_cp = ((wp_l2c_misses - cp_l2c_misses) / cp_l2c_misses) * 100

l2c_hits_wpa_cp = ((wpa_l2c_hits - cp_l2c_hits) / cp_l2c_hits) * 100
l2c_misses_wpa_cp = ((wpa_l2c_misses - cp_l2c_misses) / cp_l2c_misses) * 100

# Drop gmean
l2c_hits_wp_cp = l2c_hits_wp_cp.drop("gmean")
l2c_misses_wp_cp = l2c_misses_wp_cp.drop("gmean")

l2c_hits_wpa_cp = l2c_hits_wpa_cp.drop("gmean")
l2c_misses_wpa_cp = l2c_misses_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=l2c_hits_wp_cp.index,  # Benchmarks on the x-axis
        y=l2c_hits_wp_cp,  # Percentage difference
        name="WP Hits",
        legendgroup="WP Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l2c_hits_wpa_cp.index,  # Benchmarks on the x-axis
        y=l2c_hits_wpa_cp,  # Percentage difference
        name="WPA Hits",
        legendgroup="WPA Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l2c_misses_wp_cp.index,  # Benchmarks on the x-axis
        y=l2c_misses_wp_cp,  # Percentage difference
        name="WP Misses",
        legendgroup="WP Misses",
    )
)

fig.add_trace(
    go.Bar(
        x=l2c_misses_wpa_cp.index,  # Benchmarks on the x-axis
        y=l2c_misses_wpa_cp,  # Percentage difference
        name="WPA Misses",
        legendgroup="WPA Misses",
    )
)

fig.update_layout(
    title="L2C Cache Hits and Misses Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    # yaxis=dict(range=[0, 100]),  # Set the y-axis range from 0 to 100
)

# Show the plot
fig.show()

### LLC Hits & Misses

In [ ]:
cp_llc_hits = cp_default_df["LLC_TOTAL_HITS"]
wp_llc_hits = wp_default_df["LLC_TOTAL_HITS"]
wpa_llc_hits = wpa_default_df["LLC_TOTAL_HITS"]

cp_llc_misses = cp_default_df["LLC_TOTAL_MISS"]
wp_llc_misses = wp_default_df["LLC_TOTAL_MISS"]
wpa_llc_misses = wpa_default_df["LLC_TOTAL_MISS"]

# Compute the % change in L1I hits and misses
llc_hits_wp_cp = ((wp_llc_hits - cp_llc_hits) / cp_llc_hits) * 100
llc_misses_wp_cp = ((wp_llc_misses - cp_llc_misses) / cp_llc_misses) * 100

llc_hits_wpa_cp = ((wpa_llc_hits - cp_llc_hits) / cp_llc_hits) * 100
llc_misses_wpa_cp = ((wpa_llc_misses - cp_llc_misses) / cp_llc_misses) * 100

# Drop gmean
llc_hits_wp_cp = llc_hits_wp_cp.drop("gmean")
llc_misses_wp_cp = llc_misses_wp_cp.drop("gmean")

llc_hits_wpa_cp = llc_hits_wpa_cp.drop("gmean")
llc_misses_wpa_cp = llc_misses_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=llc_hits_wp_cp.index,  # Benchmarks on the x-axis
        y=llc_hits_wp_cp,  # Percentage difference
        name="WP Hits",
        legendgroup="WP Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=llc_hits_wpa_cp.index,  # Benchmarks on the x-axis
        y=llc_hits_wpa_cp,  # Percentage difference
        name="WPA Hits",
        legendgroup="WPA Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=llc_misses_wp_cp.index,  # Benchmarks on the x-axis
        y=llc_misses_wp_cp,  # Percentage difference
        name="WP Misses",
        legendgroup="WP Misses",
    )
)

fig.add_trace(
    go.Bar(
        x=llc_misses_wpa_cp.index,  # Benchmarks on the x-axis
        y=llc_misses_wpa_cp,  # Percentage difference
        name="WPA Misses",
        legendgroup="WPA Misses",
    )
)

fig.update_layout(
    title="LLC Cache Hits and Misses Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    # yaxis=dict(range=[0, 100]),  # Set the y-axis range from 0 to 100
)

# Show the plot
fig.show()

In [ ]:
def all_in_one(cp_df, wp_df, wpa_df, prefetcher_list):
    # Correct Path
    df_cp_l1i_default = cp.group_by(cp_df, "default")
    df_cp_l1i_default = cp.calculate_means(df_cp_l1i_default)

    df_cp_l1i_prefetchers = {}
    for key in prefetcher_list:
        if key != "default":
            prefetcher = cp.group_by(cp_df, key)
            prefetcher = cp.calculate_means(prefetcher)
            df_cp_l1i_prefetchers[key] = cp.calculate_speedup(
                df_cp_l1i_default, prefetcher
            )

    # Wrong Path
    df_wp_l1i_default = cp.group_by(wp_df, "default")
    df_wp_l1i_default = cp.calculate_means(df_wp_l1i_default)

    df_wp_l1i_prefetchers = {}
    for key in prefetcher_list:
        if key != "default":
            prefetcher = cp.group_by(wp_df, key)
            prefetcher = cp.calculate_means(prefetcher)
            df_wp_l1i_prefetchers[key] = cp.calculate_speedup(
                df_wp_l1i_default, prefetcher
            )

    # Wrong Path Aware
    df_wpa_l1i_default = cp.group_by(wpa_df, "default")
    df_wpa_l1i_default = cp.calculate_means(df_wpa_l1i_default)

    df_wpa_l1i_prefetchers = {}
    for key in prefetcher_list:
        if key != "default":
            prefetcher = cp.group_by(wpa_df, key)
            prefetcher = cp.calculate_means(prefetcher)
            df_wpa_l1i_prefetchers[key] = cp.calculate_speedup(
                df_wpa_l1i_default, prefetcher
            )

    # Plotting
    cp.create_bar_plot_per_benchmark(
        df_cp_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Correct Path (CP) | Speedup of each instruction prefetcher compared to CP FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=False,
        show_amean=False,
    ).show()

    cp.create_bar_plot_per_benchmark(
        df_wp_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Wrong Path (WP) | Speedup of each instruction prefetcher compared to WP FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=False,
        show_amean=False,
    ).show()

    cp.create_bar_plot_per_benchmark(
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Wrong Path Aware (WPA) | Speedup of each instruction prefetcher compared to WPA FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=False,
        show_amean=False,
    ).show()

    cp.create_bar_plot_gmean(
        df_cp_l1i_prefetchers,
        df_wp_l1i_prefetchers,
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Comparison of CP, WP and WPA prefetchers",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=False,
    ).show()

    # Correct Path
    df_cp_l1i_default = cp.group_by(cp_df, "default")
    df_cp_l1i_default = cp.calculate_means(df_cp_l1i_default)

    # Wrong Path
    df_wp_l1i_prefetchers = {}
    for key in prefetcher_list:
        prefetcher = cp.group_by(wp_df, key)
        prefetcher = cp.calculate_means(prefetcher)
        df_wp_l1i_prefetchers[key] = cp.calculate_speedup(df_cp_l1i_default, prefetcher)

    # Wrong Path Aware
    df_wpa_l1i_prefetchers = {}
    for key in prefetcher_list:
        prefetcher = cp.group_by(wpa_df, key)
        prefetcher = cp.calculate_means(prefetcher)
        df_wpa_l1i_prefetchers[key] = cp.calculate_speedup(
            df_cp_l1i_default, prefetcher
        )

    # Plotting
    cp.create_bar_plot_per_benchmark(
        df_wp_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Speedup of each WP instruction prefetcher compared to CP FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
        show_amean=False,
    ).show()

    cp.create_bar_plot_per_benchmark(
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Speedup of each WP instruction prefetcher compared to CP FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
        show_amean=False,
    ).show()

    cp.create_bar_plot_gmean_wp_wpa(
        df_wp_l1i_prefetchers,
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Comparison of WP and WPA prefetchers",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
    ).show()

    # Wrong Path
    df_wp_l1i_prefetchers = {}
    for key in prefetcher_list:
        cp_prefetcher = cp.group_by(cp_df, key)
        wp_prefetcher = cp.group_by(wp_df, key)

        cp_prefetcher = cp.calculate_means(cp_prefetcher)
        wp_prefetcher = cp.calculate_means(wp_prefetcher)
        df_wp_l1i_prefetchers[key] = cp.calculate_speedup(cp_prefetcher, wp_prefetcher)

    # Wrong Path Aware
    df_wpa_l1i_prefetchers = {}
    for key in prefetcher_list:
        cp_prefetcher = cp.group_by(cp_df, key)
        wpa_prefetcher = cp.group_by(wpa_df, key)

        cp_prefetcher = cp.calculate_means(cp_prefetcher)
        wpa_prefetcher = cp.calculate_means(wpa_prefetcher)
        df_wpa_l1i_prefetchers[key] = cp.calculate_speedup(
            cp_prefetcher, wpa_prefetcher
        )

    # Plotting
    cp.create_bar_plot_per_benchmark(
        df_wp_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Speedup of each WP instruction prefetcher compared to CP respective prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
        show_amean=False,
    ).show()

    cp.create_bar_plot_per_benchmark(
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Speedup of each WPA instruction prefetcher compared to CP respective prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
        show_amean=False,
    ).show()

    cp.create_bar_plot_gmean_wp_wpa(
        df_wp_l1i_prefetchers,
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Comparison of WP and WPA prefetchers",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
    ).show()

## L1I

In [ ]:
l1i_prefetchers = {
    "default": {"label": "FDIP", "color": "white", "edgecolor": "black", "hatch": "x"},
    "l1i-barsa": {"label": "Barsa", "color": "blue"},
    "l1i-bip": {"label": "BIP", "color": "green"},
    "l1i-djolt": {"label": "Djolt", "color": "orange"},
    "l1i-epi": {"label": "EPI", "color": "purple"},
    "l1i-fnlmma": {"label": "FNL-MMA", "color": "red"},
    "l1i-fnlmma_new": {"label": "FNL-MMA New", "color": "cyan"},
    "l1i-mana": {"label": "Mana", "color": "magenta"},
    "l1i-next_line": {"label": "Next Line", "color": "yellow"},
    "l1i-pips": {"label": "PIPS", "color": "lime"},
    "l1i-tap": {"label": "TAP", "color": "brown"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, l1i_prefetchers)

## L1D

In [ ]:
l1d_prefetchers = {
    "default": {"label": "None", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "l1d-berti": {"label": "Berti"},
    "l1d-ip_stride": {"label": "IP Stride"},
    "l1d-ipcp": {"label": "IPCP"},
    "l1d-next_line": {"label": "Next Line"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, l1d_prefetchers)

## L2C

In [ ]:
l2c_prefetchers = {
    "default": {"label": "None", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "l2c-ampm": {"label": "AMPM"},
    "l2c-bingo": {"label": "Bingo"},
    "l2c-bop": {"label": "BOP"},
    "l2c-dspatch": {"label": "DSPatch"},
    "l2c-ip_stride": {"label": "IP Stride"},
    "l2c-ipcp": {"label": "IPCP"},
    "l2c-mlop": {"label": "MLOP"},
    "l2c-next_line": {"label": "Next Line"},
    "l2c-sandbox": {"label": "Sandbox"},
    "l2c-scooby": {"label": "Scooby"},
    "l2c-sms": {"label": "SMS"},
    "l2c-spp_dev": {"label": "SPP-Dev"},
    "l2c-spp_ppf_dev": {"label": "SPP-PPF-Dev"},
    "l2c-streamer": {"label": "Streamer"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, l2c_prefetchers)

## LLC

In [ ]:
llc_prefetchers = {
    "default": {"label": "None", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "l2c-ip_stride": {"label": "IP Stride"},
    "l2c-next_line": {"label": "Next Line"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, llc_prefetchers)

## LLC Replacment Policy

In [ ]:
llc_replacement_policies = {
    "default": {"label": "LRU", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "ship": {"label": "SHIP"},
    "srrip": {"label": "SRRIP"},
    "drrip": {"label": "DRRIP"},
    "mockingjay": {"label": "Mockingjay"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, llc_replacement_policies)